In [1]:
from faster_rcnn.datasets.factory import get_imdb
import faster_rcnn.roi_data_layer.roidb as rdl_roidb
from faster_rcnn.roi_data_layer.layer import RoIDataLayer
from faster_rcnn.fast_rcnn.config import cfg, cfg_from_file

import cv2
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300

In [2]:
# load config
cfg_file = 'experiments/cfgs/faster_rcnn_end2end.yml'
cfg_from_file(cfg_file)

In [3]:
# imdb_name = 'kittivoc_train'
imdb_name = 'kittipose_train'

# load data
imdb = get_imdb(imdb_name)
rdl_roidb.prepare_roidb(imdb)
roidb = imdb.roidb
# roidb is a database, containing bounding box info for all training images

<bound method kittipose.default_roidb of <faster_rcnn.datasets.kittipose.kittipose object at 0x7f01ea992510>>
Remove empty annotations:  005066 004339 004040 003383 001752 001091 000547 Done. 
kittipose_train gt roidb loaded from /home/pculbert/Documents/faster_rcnn_pytorch/data/cache/kittipose_train_gt_roidb.pkl


In [ ]:
print len(roidb)# roidb is a list of dictionaries. Length is the entire number of images

In [ ]:
poses = np.concatenate([roidb[i]['poses'] for i in range(0,len(roidb))],axis=0)

In [ ]:
sig2 = np.var(poses,axis=0)
mean = np.mean(poses,axis=0)
np.var((poses-mean)/np.sqrt(sig2))

In [ ]:
print roidb[0]

img_name = roidb[0]['image'][0:-10]
print img_name

idx = roidb[0]['image'][-10:-3]
print idx

idx += "png"
print idx

disp_name = img_name + 'disparity/' + idx
print disp_name

In [4]:
data_layer = RoIDataLayer(roidb, imdb.num_classes)

In [19]:
blobs = data_layer.forward()
im_data = blobs['data'] # one image, shape = (1, 302, 1000, 3)
im_info = blobs['im_info'] # stores H, W, scale
gt_boxes = blobs['gt_boxes']
gt_ishard = blobs['gt_ishard']
dontcare_areas = blobs['dontcare_areas']
poses = blobs['gt_poses']

[ 0.13597917  0.25474813  0.3060907  -0.08941058  0.23827715 -0.26190137
  0.10365027]


In [ ]:
#print blobs['gt_poses']
#print blobs['gt_boxes']
#print blobs['im_name']

In [ ]:
plt.imshow(im_data[0,:,:,:])

In [ ]:
disp_data = blobs['data_disp']
print disp_data.shape # note: original disparity file has 3 identical channels
                      # we only take one channel
print disp_data.dtype
plt.imshow(np.uint8(disp_data[0,:,:,0])) # must convert to uint8 in order to plot

In [ ]:
# read a sample rgb image
fn = "/home/pculbert/Documents/faster_rcnn_pytorch/data/KITTIVOC/JPEGImages/000558.jpg"
im = cv2.imread(fn)
print im.shape
plt.imshow(im)

In [ ]:
# read a sample disparity image
# note that the disparity map is a 3 IDENTICAL channel rgb!
# we only need to use one channel
fn = "/home/pculbert/Documents/faster_rcnn_pytorch/data/KITTIVOC/JPEGImages/disparity/000558.png"
im = cv2.imread(fn)
print im.shape
print im.dtype
#plt.imshow(im)

im0 = im[:,:,0]
im1 = im[:,:,1]
im2 = im[:,:,2]

print np.sum(im0-im1)
print np.sum(im0-im2)

plt.imshow(im0) # only plot one channel